In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r /content/gdrive/MyDrive/essay_data/data_gcn ./

In [ ]:
with open('link') as f:
  for id in f:
    !gdown {id}
    !unzip -o data_gcn.zip
    !rm data_gcn.zip

In [ ]:
!ls -l ./data_gcn/en/* | wc -l

1790


In [ ]:
!ls -l ./data_gcn/de/* | wc -l

1790


In [ ]:
def get_content(lang, id):
  with open(f'./data_gcn/{lang}/data_{id}', encoding='utf-8', errors='ignore') as f:
    content = f.read()
  return content

In [ ]:
for lang in ['de', 'en']:
  with open(f'./data_gcn/{lang}/data.txt', 'a', encoding='utf-8', errors='ignore') as f:
    for i in range(0, 1790):
      print(f'{lang} - data_{i}')
      f.write(get_content(lang, i))

In [ ]:
!wc -l data_gcn/de/data.txt

4473746 data_gcn/de/data.txt


In [ ]:
!wc -l data_gcn/en/data.txt

4473746 data_gcn/en/data.txt


In [ ]:
import os
if not os.path.exists("raw_data_gcn"):
  !gdown 1-48YG87WajVneWP1t362CQyn_iqohqr5
  !unzip raw_data_gcn.zip
  !rm raw_data_gcn.zip

Downloading...
From: https://drive.google.com/uc?id=1-48YG87WajVneWP1t362CQyn_iqohqr5
To: /content/raw_data_gcn.zip
100% 468M/468M [00:01<00:00, 267MB/s]
Archive:  raw_data_gcn.zip
   creating: raw_data_gcn/
   creating: raw_data_gcn/.ipynb_checkpoints/
  inflating: raw_data_gcn/data.de    
  inflating: raw_data_gcn/data.en    
  inflating: raw_data_gcn/vocab.de   
  inflating: raw_data_gcn/vocab.en   


In [ ]:
raw_data_dir = 'raw_data_gcn'

# Voc2id

In [ ]:
def get_token(lang):
  filename = f'{raw_data_dir}/vocab.{lang}'
  with open(filename, encoding='utf-8', errors='ignore') as f:
    for line in f:
      token[lang].append(line.strip())
  token[lang][0] = 'UNK'      # Change <unk> token to UNK
  token[lang].remove('<s>')   # Remove <eos> token from gcn data
  token[lang].remove('</s>')  # Remove <bos> token from gcn data

def build_vocab(token, vocab):
  for i in range(len(token)):
    vocab[token[i]] = {}
    vocab[token[i]]['id'] = i
    vocab[token[i]]['freq'] = 0

In [ ]:
def voc2id(vocab, output):
  filepath = os.path.join(output, 'voc2id.txt')
  with open(filepath, 'w') as f:
    for word in vocab:
      id = vocab[word]['id']
      f.write(f'{word}\t{id}\n')

# Frequency

In [ ]:
def get_frequency():
  freq = {}
  filename = os.path.join('data_gcn', lang, 'data.txt')
  with open(filename, encoding='utf-8', errors='ignore') as f:
    for i, line in enumerate(f):
      data = line.strip().split()
      num_word = int(data[0])
      for i in range(2, num_word + 2):
        id = int(data[i])
        if id not in freq:
          freq[id] = 1 
        else:
          freq[id] += 1
  return freq

def id2freq(vocab, output):
  freq = get_frequency()
  filepath = os.path.join(output, 'id2freq.txt')
  with open(filepath, 'w') as f:
    for word in vocab:
      id = vocab[word]['id']
      f.write(f'{id}\t{freq[id] if id in freq else 0}\n')

In [ ]:
for lang in ['de', 'en']:
  id2freq(vocab[lang], os.path.join('data_gcn', lang))

# Dependencies

In [ ]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 32.8 MB/s 
     |████████████████████████████████| 4.4 MB 68.1 MB/s 
     |████████████████████████████████| 175 kB 95.5 MB/s 
     |████████████████████████████████| 101 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 69.8 MB/s 
     |████████████████████████████████| 6.6 MB 14.0 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=76cb14d323924695fc8983b7a52385a1fd268cea0edb0f6a08442bae7e9a3abd
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os
import stanza
from stanza.resources.common import DEFAULT_MODEL_DIR
from stanza.pipeline.processor import ProcessorVariant, register_processor_variant
from stanza.models.common import doc
from stanza.pipeline._constants import TOKENIZE

In [ ]:
stanza.download('en')       # This downloads the English models for the neural pipeline
stanza.download('de')

2022-06-30 15:18:23 INFO: Downloading default packages for language: en (English)...


2022-06-30 15:18:35 INFO: Finished downloading models and saved to /root/stanza_resources.


2022-06-30 15:18:36 INFO: Downloading default packages for language: de (German)...


2022-06-30 15:18:49 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
nlp = {}
nlp['en'] = stanza.Pipeline(lang='en', processors='tokenize,pos,lemma,depparse', tokenize_pretokenized=True)
nlp['de'] = stanza.Pipeline(lang='de', processors='tokenize,pos,lemma,depparse', tokenize_pretokenized=True)

2022-06-30 15:18:50 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-06-30 15:18:50 INFO: Use device: gpu
2022-06-30 15:18:50 INFO: Loading: tokenize
2022-06-30 15:18:50 INFO: Loading: pos
2022-06-30 15:19:02 INFO: Loading: lemma
2022-06-30 15:19:02 INFO: Loading: depparse
2022-06-30 15:19:03 INFO: Done loading processors!


2022-06-30 15:19:03 WARNING: Language de package default expects mwt, which has been added
2022-06-30 15:19:03 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |

2022-06-30 15:19:03 INFO: Use device: gpu
2022-06-30 15:19:03 INFO: Loading: tokenize
2022-06-30 15:19:03 INFO: Loading: mwt
2022-06-30 15:19:03 INFO: Loading: pos
2022-06-30 15:19:04 INFO: Loading: lemma
2022-06-30 15:19:04 INFO: Loading: depparse
2022-06-30 15:19:04 INFO: Done loading processors!


## Tokenizer

In [ ]:
!pip install mosestokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for mosestokenizer: filename=mosestokenizer-1.2.1-py3-none-any.whl size=49189 sha256=c45314716814b001f289324e8633e53d31421b100217828218bab7ef20e7d68f
  Stored in directory: /root/.cache/pip/wheels/b0/35/f7/af1258779a0b890abc3c79481460c597cb1f3659d0603cfb9d
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3353 sha256=7a2c49af227872a506f20c2b996fb26b7812b334e84c53c78724c54f5cd93a3c
  Stored in directory: /root/.cache/pip/wheels/c5/4f/33/54741ffe08e38ececb1d28068a153729b4fe820bafa0a0691f
  Created wheel for uctools: filename=uctools-1.3.0-py3-none-any.whl size=6161 sha256=de8f23530ff00ed47cd0b690a80f1bcd5705e087949cbd1f20cb0599b0371cf8
  Stored in directory: /root/.cache/pip/wheels/fb/44/e9/914cf8fa71f0141f9314f862538d1218fcf2b94542a0fb7d35
Successfully built mosestokenizer toolwrapper uctools


In [ ]:
# vim /usr/local/lib/python3.7/dist-packages/mosestokenizer/tokenizer-v1.1.perl
#   - @-@ -> ##AT##-##AT##
!sed -i 's/\\\@-\\\@/\\\#\\\#AT\\\#\\\#-\\\#\\\#AT\\\#\\\#/' /usr/local/lib/python3.7/dist-packages/mosestokenizer/tokenizer-v1.1.perl

In [ ]:
from mosestokenizer import *

In [ ]:
def normalize(lang, text):
  text = text.replace(' ##AT##-##AT##', '-')
  text = text.replace('&amp;', '&')
  text = text.replace('&#124;', '|')
  text = text.replace('&lt;', '<')
  text = text.replace('&gt;', '>')
  text = text.replace('&apos;', "'")
  text = text.replace('&quot;', '"')
  text = text.replace('&#91;', '[')
  text = text.replace('&#93;', ']') 
  return text 

def tokenize_step(lang, sentence):
  with MosesTokenizer(lang) as tokenize:
    return tokenize(sentence)

def split_sentence(lang, sentence):
  with MosesSentenceSplitter(lang) as splitsents:
      return splitsents([sentence])

def punctuation_normalize(lang, sentence):
  with MosesPunctuationNormalizer(lang) as normalize:
      return normalize(sentence)

def sequence_transform(*steps):
  def func(lang, input):
    for step in steps:
      input = step(lang, input)
    return input
  return func

tokenizer = sequence_transform(normalize, 
                              punctuation_normalize,
                              tokenize_step)

In [ ]:
import os

def get_deprel(dep):
  with open('/content/gdrive/MyDrive/dep.txt', encoding='utf-8', errors='ignore') as f:
    counter = 0
    for line in f:
      dep[line.strip()] = counter;
      counter+=1

def de2id(dep, output):  
  filepath = os.path.join(output, 'de2id.txt')
  with open(filepath, 'w') as f:
    f.write('UNK\t-1\n')
    f.writelines([f'{deprel}\t{dep[deprel]}\n' for deprel in  dep])
    
def get_dep(dep, text):
  if text not in dep:
    dep[text] = len(dep)
  return dep[text]

def dep_process(lang, sentence, new_dep):
  print(sentence)
  doc = nlp[lang]([sentence])
  count = 0
  for sent in doc.sentences:
    for i, word in enumerate(sent.words):
      if word.deprel not in dep[lang]:
        print(f'{word.deprel}\t{new_dep[count]}')
        dep[lang][word.deprel] = new_dep[count]
        count+=1

def add_more_dep(lang):
  filename = os.path.join('data_gcn', lang, 'data.txt')
  with open(filename, encoding='utf-8', errors = 'ignore') as f:
    with open(f'raw_data_gcn/data.{lang}', encoding='utf-8', errors = 'ignore') as f_raw:
      counter = 0
      for line, line_raw in zip(f, f_raw):
        counter += 1
        if counter % 100000 == 0:
          print(counter)
        data = line.strip().split()
        num_word = int(data[0])
        num_dep = int(data[1])
        new_dep = []
        for i in range(num_word + 2, len(data)):
          deprel = data[i].split('|')[2]
          if int(deprel) not in dep[lang].values():
            new_dep.append(deprel)
        if new_dep:
          print(counter)
          print(new_dep)
          dep_process(lang, tokenizer(lang, line_raw), new_dep)

In [ ]:
token = {}
vocab = {}
dep = {}
for lang in ['de', 'en']:
  token[lang] = []
  vocab[lang] = {}
  dep[lang] = {}

In [ ]:
for lang in ['en', 'de']:
  # Get vocab and dep
  output_dir = os.path.join('data_gcn', lang)
  get_token(lang)

  build_vocab(token[lang], vocab[lang])

  get_deprel(dep[lang])

  add_more_dep(lang)

  # voc2id
  print('- voc2id')
  voc2id(vocab[lang], output_dir)

  #de2id
  print('- de2id') 
  de2id(dep[lang], output_dir)

  #id2freq
  print('- id2freq')
  id2freq(vocab[lang], output_dir)

# Final

In [ ]:
!rm -rf data_gcn/**/data_*

In [ ]:
!zip -r data_gcn.zip data_gcn
!cp -r data_gcn /content/gdrive/MyDrive/essay_data/
!cp data_gcn.zip /content/gdrive/MyDrive/essay_data/

  adding: data_gcn/ (stored 0%)
  adding: data_gcn/en/ (stored 0%)
  adding: data_gcn/en/data.txt (deflated 68%)
  adding: data_gcn/en/voc2id.txt (deflated 51%)
  adding: data_gcn/en/de2id.txt (deflated 43%)
  adding: data_gcn/en/id2freq.txt (deflated 70%)
  adding: data_gcn/de/ (stored 0%)
  adding: data_gcn/de/data.txt (deflated 67%)
  adding: data_gcn/de/voc2id.txt (deflated 53%)
  adding: data_gcn/de/de2id.txt (deflated 43%)
  adding: data_gcn/de/id2freq.txt (deflated 69%)
